In [19]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import xgboost as xgb
from scipy.stats import shapiro

In [20]:
dados = pd.read_csv('kc_house_data.csv')

In [21]:
# Filtrando os dados com preço acima de 2.000.000
dados_acima_2milhoes = dados[dados['price'] < 2000000]

In [22]:
#Eliminando casas com 5 quartos ou mais
dados2 = dados.drop(dados[dados['bedrooms'] > 5].index)

In [23]:
#Eliminando casas com 5 banheiros ou menos
dados3 = dados2.drop(dados2[dados2['bathrooms'] > 5].index)

In [24]:
#Eliminando casas onde o sqft_living sejam maiores que 5000
dados4 = dados3.drop(dados3[dados3['sqft_living'] > 5000].index)

In [25]:
#Eliminando casas com mais de 3 andares 
dados5 = dados4.drop(dados4[dados4['floors'] > 3].index)

In [26]:
dados6 = dados5[dados5['price'] <1500000]

In [27]:
#Removendo Colunas 

dados7 = dados6.copy()
colunas_para_remover = ['waterfront', 'view', 'zipcode', 'lat', 'long']
dados7 = dados7.drop(columns=colunas_para_remover)

In [28]:
dados8= dados7.copy()

# Converta a coluna 'date' para datetime
dados8['date'] = pd.to_datetime(dados8['date'], format='%Y%m%dT%H%M%S')

dados8['yr_renovated'] = dados8['yr_renovated'].replace(0, pd.NA)

# Crie uma nova coluna para armazenar a diferença entre os anos
dados8['diferenca_anos'] = dados8['date'].dt.year - dados8['yr_renovated']

dados8['diferenca_anos'].fillna(0, inplace=True)

# Exclua a coluna 'yr_renovated'
dados8.drop('yr_renovated', axis=1, inplace=True)



C:\Users\Sávio\AppData\Local\Temp\ipykernel_12016\2865857541.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dados8['diferenca_anos'].fillna(0, inplace=True)
C:\Users\Sávio\AppData\Local\Temp\ipykernel_12016\2865857541.py:11: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dados8['diferenca_anos'].fillna(0, 

In [29]:
# Calcule a idade da casa
dados8['idade_casa'] = dados8['date'].dt.year - dados8['yr_built']

In [30]:
dados8.drop(['date'], axis=1, inplace=True)

In [31]:
dados8.set_index('id', inplace=True)

In [32]:
features = dados8.drop(columns=['price', 'yr_built'])  # Todas as colunas exceto 'price'
target = dados8['price']  # A coluna 'price'

In [33]:
# Criar um objeto MinMaxScaler
scaler = MinMaxScaler()

# Normalizar as variáveis do conjunto de treino
X_normalized = scaler.fit_transform(features)


In [34]:
with open('random_forest_model.pkl', 'rb') as arquivo:
    modelo_carregado = pickle.load(arquivo)
previsoes = modelo_carregado.predict(X_normalized)

In [35]:
dados7['Previsoes'] = previsoes
dados7

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,sqft_living15,sqft_lot15,Previsoes
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,3,7,1180,0,1955,0,1340,5650,265126.00
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,3,7,2170,400,1951,1991,1690,7639,567194.50
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,3,6,770,0,1933,0,2720,8062,233736.80
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,5,7,1050,910,1965,0,1360,5000,437482.30
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,3,8,1680,0,1987,0,1800,7503,478413.56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21608,263000018,20140521T000000,360000.0,3,2.50,1530,1131,3.0,3,8,1530,0,2009,0,1530,1509,466915.50
21609,6600060120,20150223T000000,400000.0,4,2.50,2310,5813,2.0,3,8,2310,0,2014,0,1830,7200,422351.10
21610,1523300141,20140623T000000,402101.0,2,0.75,1020,1350,2.0,3,7,1020,0,2009,0,1020,2007,376335.14
21611,291310100,20150116T000000,400000.0,3,2.50,1600,2388,2.0,3,8,1600,0,2004,0,1410,1287,391593.20


In [36]:
df_selecionado = dados7[['price', 'bathrooms', 'bedrooms', 'Previsoes']]
df_selecionado

,price,bathrooms,bedrooms,Previsoes
0,221900.0,1.00,3,265126.00
1,538000.0,2.25,3,567194.50
2,180000.0,1.00,2,233736.80
3,604000.0,3.00,4,437482.30
4,510000.0,2.00,3,478413.56
...,...,...,...,...
21608,360000.0,2.50,3,466915.50
21609,400000.0,2.50,4,422351.10
21610,402101.0,0.75,2,376335.14
21611,400000.0,2.50,3,391593.20
